In [45]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.models import load_model

In [46]:
# Creating a function to train the model
def get_model():
    df = pd.read_csv('gold_price.csv')

    # Convert the date column to datetime type and set it as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Scaling the price data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_price = scaler.fit_transform(df['Price'].values.reshape(-1, 1))

    # Creating a function to create the LSTM model
    def create_lstm_model():
        model = Sequential()
        model.add(LSTM(units=80, return_sequences=True, input_shape=(60, 1)))
        model.add(LSTM(units=80))
        model.add(Dense(units=1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model


    # Creating the training data
    x_train, y_train = [], []
    for i in range(60, scaled_price.shape[0]):
        x_train.append(scaled_price[i-60:i, 0])
        y_train.append(scaled_price[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshaping the data for the LSTM model
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Creating the LSTM model
    model = create_lstm_model()

    # Training the LSTM model
    model.fit(x_train, y_train, epochs=10, batch_size=25)

    # Creating the testing data
    test_data = scaled_price[-60:]
    x_test = []
    x_test.append(test_data)
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Saving the model
    model.save('gold_model.h5')

In [47]:
# Creating a function to load the model and predict the price
def predict_price():
    # Checking if the model is already trained
    try:
        model = load_model('gold_model.h5')
    except:
        get_model('gold_price.csv')
        model = load_model('gold_model.h5')

    df = pd.read_csv('gold_price.csv')

    # Convert the date column to datetime type and set it as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Scaling the price data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_price = scaler.fit_transform(df['Price'].values.reshape(-1, 1))

    # Creating the testing data
    test_data = scaled_price[-60:]
    x_test = []
    x_test.append(test_data)
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Predicting the prices using the LSTM model for the next 30 days
    predicted_prices = []
    for i in range(30):
        predicted_price = model.predict(x_test)
        predicted_prices.append(predicted_price[0, 0])
        x_test = np.concatenate(
            (x_test[:, 1:, :], predicted_price.reshape(1, 1, 1)), axis=1)

    # Scaling the predicted prices back to their original range
    predicted_prices = scaler.inverse_transform(
        np.array(predicted_prices).reshape(-1, 1))

    # Converting the predicted prices to a list
    predicted_prices = predicted_prices.reshape(1, -1)[0].tolist()
    return predicted_prices

In [49]:
predictions =  predict_price()

print(len(predictions))

2023-04-12 16:05:16.692634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-12 16:05:16.695527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-12 16:05:16.697405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 33ms/step
30
